In [1]:
import threading
import time
import webbrowser
from http.server import HTTPServer, BaseHTTPRequestHandler
from urllib.parse import urlparse, parse_qs
import requests
from urllib.parse import quote
import json

### for datasphere cli

In [18]:
CLIENT_ID = "sb-a0aa5cb4-c429-44b3-a601-531022f35a68!b240071|client!b3650"
REDIRECT_URI = "http://localhost:8080/"

AUTHORIZE = (
    "https://indaver-dwc-q.authentication.eu10.hana.ondemand.com/oauth/authorize"
    "?response_type=code"
    f"&client_id={quote(CLIENT_ID)}"
    f"&redirect_uri={quote(REDIRECT_URI)}"
)

class OAuthHandler(BaseHTTPRequestHandler):
    """Minimal one-shot HTTP server to catch the redirect."""
    def do_GET(self):
        qs = parse_qs(urlparse(self.path).query)
        self.server.code = qs.get("code", [None])[0]
        # Nice message in the browser
        self.send_response(200)
        self.end_headers()
        self.wfile.write(b"You may close this tab.")
    def log_message(self, *args):  # silence console
        return

# 1) Start server in background
server = HTTPServer(("localhost", 8080), OAuthHandler)
threading.Thread(target=server.serve_forever, daemon=True).start()

# 2) Open browser so user can log in
webbrowser.open(AUTHORIZE)

# 3) Wait for the redirect with the auth code
while not hasattr(server, "code"):
    time.sleep(0.2)

auth_code = server.code
print("Auth-code =", auth_code)
server.shutdown()

Auth-code = 3MbrFHOVXPz7e0N9sbsZLTM9yqMqhBYo


In [19]:
CLIENT_SECRET = "514f5084-d874-4fe8-8f85-a0f13fc4862b$N-bF8f34CHcekDgwFEXjayPSr6AUJrzjA1HlwC2eanc="  # Zet dit in je omgeving
CODE = auth_code

TOKEN_URL = "https://indaver-dwc-q.authentication.eu10.hana.ondemand.com/oauth/token"

response = requests.post(
    TOKEN_URL,
    data={
        "grant_type": "authorization_code",
        "code": CODE,
        "redirect_uri": REDIRECT_URI
    },
    auth=(CLIENT_ID, CLIENT_SECRET),
    headers={"Accept": "application/json"}
)

print(response.status_code)

200


In [20]:
print(response.json()["access_token"])

eyJhbGciOiJSUzI1NiIsImprdSI6Imh0dHBzOi8vaW5kYXZlci1kd2MtcS5hdXRoZW50aWNhdGlvbi5ldTEwLmhhbmEub25kZW1hbmQuY29tL3Rva2VuX2tleXMiLCJraWQiOiJkZWZhdWx0LWp3dC1rZXktLTIwODgwMjg5NTMiLCJ0eXAiOiJKV1QiLCJqaWQiOiAiSGoxMkd6Rko2T3NFZ0MvLzFqZm5zaGQvUm9HdmlKTjR0ZERKRkYwbURXND0ifQ.eyJqdGkiOiJmOThmYzI3YTFlYWM0YmQyYWNiZTNjZTI5MTgwYjU2OSIsImV4dF9hdHRyIjp7ImVuaGFuY2VyIjoiWFNVQUEiLCJzdWJhY2NvdW50aWQiOiJhNjkzNzFiYy1hZDQ5LTRkZjgtOGU5Ny1mOTlmZWRjNDUyYzUiLCJ6ZG4iOiJpbmRhdmVyLWR3Yy1xIiwic2VydmljZWluc3RhbmNlaWQiOiJiOGE3NjhlZC1hNWRiLTQ4YzItODI0Yi0wODgwMTEyZTJjYTkifSwieHMuc3lzdGVtLmF0dHJpYnV0ZXMiOnsieHMuc2FtbC5ncm91cHMiOlsic2FjIl0sInhzLnJvbGVjb2xsZWN0aW9ucyI6WyJzYWMudXNlcnMiXX0sImdpdmVuX25hbWUiOiJzdGVmLmRlcHJhZXRlcmUiLCJ4cy51c2VyLmF0dHJpYnV0ZXMiOnt9LCJmYW1pbHlfbmFtZSI6ImZsZXhzby5jb20iLCJzdWIiOiJlMDM2Mjg0MC0xNDcyLTRiZGMtODZiNi1hOThhNzIzZTZkM2EiLCJzY29wZSI6WyJvcGVuaWQiLCJhcHByb3V0ZXItc2FjLXNhY2V1MTAhdDM2NTAuc2FwLmZwYS51c2VyIiwidWFhLnVzZXIiXSwiY2xpZW50X2lkIjoic2ItYTBhYTVjYjQtYzQyOS00NGIzLWE2MDEtNTMxMDIyZjM1YTY4IWIyNDAwN

In [21]:
print(response.json()["refresh_token"])

1f1fb0c83d864826b29e7abdd407ce03-r


### for sap ai core

In [3]:
ai_core_secrets_path = "ai_core_secret_orchestration.json"

In [4]:
# --- Load secrets ---
with open(ai_core_secrets_path, encoding="utf-8") as f:
    secrets = json.load(f)
AUTH_URL      = secrets["auth_url"]
CLIENT_ID     = secrets["client_id"]
CLIENT_SECRET = secrets["client_secret"]

In [9]:
resp = get_sap_token()

In [17]:
resp.json()["expires_in"]

43199

In [16]:
for key in resp.json():
    print(key)

access_token
token_type
expires_in
scope
jti
